In [1]:
from treble_tsdk.tsdk import TSDK, TSDKCredentials
from treble_tsdk import display_data as dd
from treble_tsdk import treble
import random
import json
from tqdm import tqdm
import glob
import math
from typing import List, Tuple
from collections import defaultdict

In [32]:
data_dir = "data"

rooms_data = []
for filename in tqdm(glob.glob(f"{data_dir}/**/*.json", recursive=True), desc="Loading room data"):
    with open(filename, "r") as f:
        data = json.load(f)
        room_name = filename.split("/")[-2]
        data["room_name"] = room_name
        rooms_data.append(data)

tsdk = TSDK(TSDKCredentials.from_file("./creds/tsdk.cred"))


Loading room data: 100%|██████████| 3/3 [00:00<00:00, 1414.76it/s]


== SDK package is up to date ==


In [33]:
project = tsdk.get_or_create_project("ertsi_11")

In [34]:
def sort_points_clockwise(points: List[List[float]]) -> List[List[float]]:
    # Compute centroid
    centroid_x = sum(x for x, y in points) / len(points)
    centroid_y = sum(y for x, y in points) / len(points)
    
    # Function to compute angle from centroid
    def angle_from_centroid(point):
        x, y = point
        return math.atan2(y - centroid_y, x - centroid_x)
    
    # Sort points by angle (clockwise)
    return sorted(points, key=angle_from_centroid, reverse=True)

In [35]:
room_definitions = []
for i, room_data in enumerate(rooms_data):
    edge_points_hexagon = sort_points_clockwise(room_data["room_verts"]) if not "sorted" in room_data else room_data["room_verts"]
    room_height = 3
    print(room_data["room_name"], edge_points_hexagon)
    room = treble.GeometryDefinitionGenerator.create_polygon_room(
        points_xy=edge_points_hexagon, height_z=room_height, join_wall_layers=True
    )

    room_definitions.append(room)
room_definitions

jojo_8_unsorted [[-15.351582527160645, 3.730515480041504], [-11.191547393798828, 11.191547393798828], [11.191547393798828, 11.191547393798828], [15.351582527160645, 3.730515956878662], [15.351582527160645, -3.730515480041504], [11.191547393798828, -11.191547393798828], [-11.191547393798828, -11.191547393798828], [-15.351582527160645, -3.730515956878662]]
jojo_7_sorted [[1.0, -1.0], [1.0, 10.0], [-14.0, 10.0], [-14.0, 4.5], [-7.0, 4.5], [-7.0, -1.0]]
jojo_6_unsorted [[-10.0, 18.0], [10.0, 18.0], [10.0, -1.0], [-10.0, -1.0]]


[GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.]

In [36]:
generated_rooms = []

print("=== Populating rooms with geometry components ===")
for i, (room_def, room_data) in tqdm(enumerate(zip(room_definitions, rooms_data))):
    print(f"Populating room {room_data["room_name"]} with geometry components")
    room_def.clear_geometry_components()
    placements = []

    if room_data["chair_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
                components=tsdk.geometry_component_library.query(group="chair"),
            preferred_count=room_data["chair_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
    if room_data["desk_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
               components=tsdk.geometry_component_library.query(group="desk"),
            preferred_count=room_data["desk_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
        
    room_def.populate_with_geometry_components(
        components=placements,
        selection_algorithm=treble.ComponentSelectionAlgorithm.random,
    )

    try:
        model = project.add_model(f"room_{i}", room_def)
        generated_rooms.append(model)
    except Exception as e:
        print(f"Error adding model for room {room_data["room_name"]}: {e}")
        continue

=== Populating rooms with geometry components ===


0it [00:00, ?it/s]

Populating room jojo_8_unsorted with geometry components


1it [00:05,  5.67s/it]

Populating room jojo_7_sorted with geometry components


2it [00:09,  4.31s/it]

Populating room jojo_6_unsorted with geometry components


3it [00:13,  4.46s/it]


In [41]:
random_model = random.choice(generated_rooms)
random_model.plot()

In [42]:
dd.display(generated_rooms)

                                                 Models                                                 
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index ┃ Name   ┃ Id                                   ┃ Created At                   ┃ Is Watertight ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 0     │ room_0 │ 422a2822-83e4-45c1-a0dc-b9cc5d0e22fc │ 2025-05-28T14:36:51.2104487Z │ False         │
│ 1     │ room_1 │ 63e2c151-e42e-4c10-b158-3134ede9469e │ 2025-05-28 14:36:54          │ True          │
│ 2     │ room_2 │ 9cd0afff-2ddd-4861-8a06-febe57cbd96e │ 2025-05-28 14:36:58          │ True          │
└───────┴────────┴──────────────────────────────────────┴──────────────────────────────┴───────────────┘

In [43]:
all_materials = tsdk.material_library.get()
database_materials = [
    material for material in all_materials if material["organizationId"] == None
]

all_material_assignments = []
for i, (model, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    layers = {
        "polygon_room_walls": "gypsum/plaster on solid backing",
        "polygon_room_floor": room_data["floor_material"],
        "polygon_room_ceiling": room_data["ceiling_material"],
        "Furniture/Desk": "wood",
        "Furniture/Chair": "upholstered concert chairs",
        "Furniture/Chair A": "upholstered concert chairs",
        "Furniture/Bar Stool": "upholstered concert chairs",
    }
    material_assignment = []

    for layer in model.layer_names:
        if layer in layers:
            search_string = layers[layer]
            matches = [
                m for m in database_materials if search_string.lower() in m.name.lower()
            ]
            if matches:
                material_assignment.append(
                    treble.MaterialAssignment(layer, random.choice(matches))
                )
    all_material_assignments.append(material_assignment)

dd.display(all_material_assignments[-1])

3it [00:00, 23.28it/s]


                     Material assignment                     
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer name           ┃ Material name                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ polygon_room_floor   │ Linoleum or vinyl on concrete      │
│ Furniture/Bar Stool  │ Heavily upholstered concert chairs │
│ Furniture/Chair      │ Heavily upholstered concert chairs │
│ Furniture/Desk       │ Plywood panelling, 1 cm thick      │
│ polygon_room_ceiling │ Smooth painted concrete            │
│ Furniture/Chair A    │ Heavily upholstered concert chairs │
│ polygon_room_walls   │ Gypsum/Plaster on solid backing    │
└──────────────────────┴────────────────────────────────────┘

In [44]:
all_sources = []
all_source_positions = []
all_receivers = []
all_receiver_positions = []

pg = treble.PointsGenerator()

pos_ruleset = treble.PointRuleset(
    min_dist_from_surface=0.5,
    min_dist_from_other_points=2,
)

for room in tqdm(generated_rooms):
    pos = pg.generate_valid_points(
        model=room,
        max_count=2,	
        ruleset=pos_ruleset,
        z_range=(0.5, 1.5)
    )
    all_source_positions.append(pos[0])
    all_receiver_positions.append(pos[1])

    source = treble.Source.make_omni(
        position=pos[0],
        label=f"source_{room.name}",
    )
    receiver = treble.Receiver.make_mono(
        position=pos[1],
        label=f"receiver_{room.name}",
    )
    all_sources.append(source)
    all_receivers.append(receiver)


100%|██████████| 3/3 [00:01<00:00,  2.17it/s]


In [45]:
sim_type = treble.SimulationType.geometrical
crossover_frequency = 500

sim_defs = []

for i, (room, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    sim_def = treble.SimulationDefinition(
        name=f"simulation_{i}_1",  # unique name of the simulation
        simulation_type=sim_type,  # the type of simulation
        # crossover_frequency=crossover_frequency,  # the frequency at which the simulation switches from wavebased to image source
        model=room,
        energy_decay_threshold=60,  # simulation termination criteria - the simulation stops running after -60 dB of energy decay
        receiver_list=[all_receivers[i]],
        source_list=[all_sources[i]],
        material_assignment=all_material_assignments[i],
    )
    sim_defs.append(sim_def)

sim_defs = project.add_simulations(sim_defs)

3it [00:00, 3084.05it/s]


In [46]:
all_sims = project.get_simulations()
dd.display(all_sims)

                                                      Simulations                                                      
┏━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━┓
┃ Index ┃ Name           ┃ Created at       ┃ Id                                   ┃ Status  ┃ Freq. (Hz) ┃ Rcv ┃ Src ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━┩
│ 0     │ simulation_0_1 │ 2025-05-28 14:37 │ eafc80e3-c908-496e-b84e-d8216aa62db7 │ Created │ 25         │ 1   │ 1   │
│ 1     │ simulation_2_1 │ 2025-05-28 14:37 │ 4db48efb-a6de-4fe3-993f-9eac790a23c3 │ Created │ 25         │ 1   │ 1   │
│ 2     │ simulation_1_1 │ 2025-05-28 14:37 │ 8135e4aa-db4d-4d66-9767-3d27195b6480 │ Created │ 25         │ 1   │ 1   │
└───────┴────────────────┴──────────────────┴──────────────────────────────────────┴─────────┴────────────┴─────┴─────┘

In [47]:
random_sim = random.choice(all_sims)
random_sim.plot()

In [48]:
runtime_estimate = project.estimate()
dd.display(runtime_estimate)


                               Project 20f6db10-5167-4702-83ec-44e96ff7ab38 estimate                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                                                       ┃       Simulation ┃ Estimated runtime ┃   Estimated cost ┃
┃                                            Simulation ┃           status ┃           (hours) ┃         (tokens) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ simulation_0_1 (eafc80e3-c908-496e-b84e-d8216aa62db7) │          Created │             0.061 │            0.061 │
│ simulation_2_1 (4db48efb-a6de-4fe3-993f-9eac790a23c3) │          Created │             0.038 │            0.038 │
│ simulation_1_1 (8135e4aa-db4d-4d66-9767-3d27195b6480) │          Created │             0.032 │            0.032 │
├───────────────────────────────────────────────────────┼──────────────────┼───────────────────┼──────────────────┤
│                                                 Total │                  │             0.061 │            0.131 │
└───────────────────────────────────────────────────────┴──────────────────┴───────────────────┴──────────────────┘

In [49]:
project.start_simulations()
project.as_live_progress()

                                     ertsi_11                                      
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Simulation Name ┃ Id                                   ┃ Status    ┃ Progress % ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ simulation_2_1  │ 4db48efb-a6de-4fe3-993f-9eac790a23c3 │ Completed │ 100%       │
│ simulation_1_1  │ 8135e4aa-db4d-4d66-9767-3d27195b6480 │ Completed │ 100%       │
│ simulation_0_1  │ eafc80e3-c908-496e-b84e-d8216aa62db7 │ Completed │ 100%       │
├─────────────────┼──────────────────────────────────────┼───────────┼────────────┤
│                 │                                      │ Completed │ 100%       │
└─────────────────┴──────────────────────────────────────┴───────────┴────────────┘

Done

In [50]:
for sim, room in zip(all_sims, rooms_data):
    room_name = room["room_name"]
    
    try:
        r = sim.download_results(f'data/{room_name}')
    except ValueError as e:
        print(f"Room {room_name} already exists, skipping download.")
        r = None

Download simulation eafc80e3-c908-496e-b84e-d8216aa62db7: 100%|██████████| 2/2 tasks
Download simulation 4db48efb-a6de-4fe3-993f-9eac790a23c3:   0%|          | 0/2 tasks

Download simulation 4db48efb-a6de-4fe3-993f-9eac790a23c3: 100%|██████████| 2/2 tasks
Download simulation 8135e4aa-db4d-4d66-9767-3d27195b6480: 100%|██████████| 2/2 tasks


In [51]:
for simulation, room in zip(all_sims, rooms_data):
    try:
        results = simulation.get_results_object(f'data/{room["room_name"]}')
        if not results:
            print(f"No results found for room {room['room_name']}")
            continue
        mono_ir = results.get_mono_ir(source=simulation.sources[0], receiver=simulation.receivers[0])
        mono_ir.write_to_wav(path_to_file=f"data/{room['room_name']}/ir.wav")
    except KeyError as e:
        print(f"Results not found for room {room['room_name']}: {e}")
